Import, install necessary packages

In [0]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 4.9MB/s 


In [0]:
import pandas as pd
import numpy as np
import collections
import itertools

# Creating the model
from keras.layers import Bidirectional, CuDNNLSTM, Dense, Dropout, TimeDistributed
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences

# Results analysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score

import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case= False)

Using TensorFlow backend.


100%|██████████| 213450/213450 [00:00<00:00, 1130228.31B/s]


# **Get data**


In [0]:
train_url = 'https://raw.githubusercontent.com/cicl-iscl/CyberWallE/master/data/train-data-improved-sentiwordnet-arguingfull-pos.tsv?token=AFDEFD2TTBVOXQZ6HVMFGMC6DR6RM'
train_df = pd.read_csv(train_url,sep='\t', header=4, quoting = 3)

In [0]:
train_df.head()

,document_id,sent_id,token_start,token_end,token,label,positive,negative,arglex,ADJ,ADP,ADV,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SYM,VERB,X
0,111111111,1,0,4,Next,O,0.000000,0.031250,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,111111111,1,5,11,plague,O,0.071429,0.214286,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,111111111,1,12,20,outbreak,O,0.000000,0.125000,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,111111111,1,21,23,in,O,0.000000,0.000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,111111111,1,24,34,Madagascar,O,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [0]:
train_df.shape

(401288, 24)

# **Get sentences**

In [0]:
def get_sents(train_df):
  df_sents = train_df.groupby('sent_id')['token'].apply(list)
  df_sents = df_sents.to_frame()
  df_sents['sent_id'] = df_sents.index
  df_sents["sentences"]= df_sents["token"].str.join(" ")
  return df_sents

In [0]:
df_sents = get_sents(train_df)
df_sents.head()

,token,sent_id,sentences
sent_id,,,
1,"[Next, plague, outbreak, in, Madagascar, could...",1,Next plague outbreak in Madagascar could be ' ...
2,"[Geneva, -, The, World, Health, Organisation, ...",2,Geneva - The World Health Organisation chief o...
3,"["", The, next, transmission, could, be, more, ...",3,""" The next transmission could be more pronounc..."
4,"[An, outbreak, of, both, bubonic, plague, ,, w...",4,"An outbreak of both bubonic plague , which is ..."
5,"[has, killed, more, than, 200, people, in, the...",5,has killed more than 200 people in the Indian ...


In [0]:
def get_sents_as_arrays(df_sents):
  """Get one list of tokens per sentence, corresponding to the tokens in that sentence."""
  token_lists = df_sents["token"].tolist()  
  return token_lists

def normalize_sents(MAX_SEQ_LEN, token_lists):
  """Cap all token lists at max sequence length tokens and join them into sentence strings"""
  merge_test_text_normalized = []
  for sent in token_lists:
    if len(sent) > MAX_SEQ_LEN:
      merge_test_text_normalized.append(sent[:MAX_SEQ_LEN])
    else:
      merge_test_text_normalized.append(sent)

  normalized_sent_list = []
  for i in merge_test_text_normalized:
    normalized_sent_list.append(" ".join(str(j) for j in i))

  return normalized_sent_list

In [0]:
MAX_SEQ_LEN = 35

In [0]:
token_lists = get_sents_as_arrays(df_sents)
normalized_sent_list = normalize_sents(MAX_SEQ_LEN, token_lists)

print(token_lists[0:5])
print(normalized_sent_list[0:5])

[['Next', 'plague', 'outbreak', 'in', 'Madagascar', 'could', 'be', "'", 'stronger', "'", ':', 'WHO'], ['Geneva', '-', 'The', 'World', 'Health', 'Organisation', 'chief', 'on', 'Wednesday', 'said', 'a', 'deadly', 'plague', 'epidemic', 'appeared', 'to', 'have', 'been', 'brought', 'under', 'control', 'in', 'Madagascar', ',', 'but', 'warned', 'the', 'next', 'outbreak', 'would', 'likely', 'be', 'stronger', '.'], ['"', 'The', 'next', 'transmission', 'could', 'be', 'more', 'pronounced', 'or', 'stronger', ',', '"', 'WHO', 'Director', '-', 'General', 'Tedros', 'Adhanom', 'Ghebreyesus', 'told', 'reporters', 'in', 'Geneva', ',', 'insisting', 'that', '"', 'the', 'issue', 'is', 'serious', '.', '"'], ['An', 'outbreak', 'of', 'both', 'bubonic', 'plague', ',', 'which', 'is', 'spread', 'by', 'infected', 'rats', 'via', 'flea', 'bites', ',', 'and', 'pneumonic', 'plague', ',', 'spread', 'person', 'to', 'person', ','], ['has', 'killed', 'more', 'than', '200', 'people', 'in', 'the', 'Indian', 'Ocean', 'islan

#**Process sentences as needed for BERT**

BERT requires:


*   Tokenized sentences with beginning and end markers ([CLS] and [SEP])
*   Tokens mapped to their integer ID's within BERT's vocabulary, padded to the max sequence length
*   Attention masks of 1's for tokens and 0's for padding




In [0]:
def add_bert_special_tokens_and_tokenize(normalized_sent_list):
  bert_tokenized_sent_list = []
  bert_indexed_tokens_list = []
  for sent in normalized_sent_list:
    sent = "[CLS] " + sent + " [SEP]"
    tokenized_text = tokenizer.tokenize(sent)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    bert_tokenized_sent_list.append(tokenized_text)
    bert_indexed_tokens_list.append(indexed_tokens)

  return bert_tokenized_sent_list, bert_indexed_tokens_list

In [0]:
bert_preprocessed_sents = add_bert_special_tokens_and_tokenize(normalized_sent_list)

In [0]:
bert_tokenized_sent_list = bert_preprocessed_sents[0]
bert_indexed_tokens_list = bert_preprocessed_sents[1]

print(bert_tokenized_sent_list[0])
print(bert_indexed_tokens_list[0])

['[CLS]', 'Next', 'plague', 'outbreak', 'in', 'Madagascar', 'could', 'be', "'", 'stronger', "'", ':', 'WHO', '[SEP]']
[101, 5893, 13824, 8010, 1107, 12014, 1180, 1129, 112, 5992, 112, 131, 23750, 102]


Padding 

source: https://mccormickml.com/2019/07/22/BERT-fine-tuning/#32-sentences-to-ids

In [0]:
def pad_to_max_len(MAX_SEQ_LEN, bert_indexed_tokens_list):
  print('\nPadding/truncating all sentences to %d values...' % MAX_SEQ_LEN)

  bert_indexed_tokens_list = pad_sequences(bert_indexed_tokens_list, maxlen=MAX_SEQ_LEN, dtype="long", 
                            value=0, truncating="post", padding="post")
  print('\nDone.')
  return bert_indexed_tokens_list

In [0]:
bert_indexed_tokens_list = pad_to_max_len(MAX_SEQ_LEN, bert_indexed_tokens_list)


Padding/truncating all sentences to 35 values...

Done.


In [0]:
bert_indexed_tokens_list[0:4]

array([[  101,  5893, 13824,  8010,  1107, 12014,  1180,  1129,   112,
         5992,   112,   131, 23750,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0],
       [  101,  9571,   118,  1109,  1291,  3225, 14213,  2705,  1113,
         9031,  1163,   170, 10310, 13824, 18162,  1691,  1106,  1138,
         1151,  1814,  1223,  1654,  1107, 12014,   117,  1133,  6926,
         1103,  1397,  8010,  1156,  2620,  1129,  5992,   119],
       [  101,   107,  1109,  1397,  6580,  1180,  1129,  1167,  8481,
         1137,  5992,   117,   107, 23750,  2524,   118,  1615,  6564,
         5864, 24930,  3822,  4165,   144,  4638,  9730, 25014,  1361,
         1500, 13509,  1107,  9571,   117, 25504,  1115,   107],
       [  101,  1760,  8010,  1104,  1241,   171, 10354, 13207, 13824,
          117,  1134,  1110,  2819,  1118, 10594, 13475,  2258, 22593,
         4490, 17905,   

Add attention masks to tell BERT what is a token and what is padding

In [0]:
def add_attention_masks(bert_indexed_tokens_list): 
  attention_masks = []
  for sent in bert_indexed_tokens_list:
      #bool True cast to int is 1
      att_mask = [int(token_id > 0) for token_id in sent]
      attention_masks.append(att_mask)
  return attention_masks

In [0]:
bert_indexed_tokens_list_first = bert_indexed_tokens_list[:len(bert_indexed_tokens_list)//2]
bert_indexed_tokens_list_second = bert_indexed_tokens_list[len(bert_indexed_tokens_list)//2:]
print(len(bert_indexed_tokens_list_first))
print(len(bert_indexed_tokens_list_second))

10750
10751


In [0]:
# attention_masks = add_attention_masks(bert_indexed_tokens_list)
# print(attention_masks[0:4])

In [0]:
attention_masks_1 = add_attention_masks(bert_indexed_tokens_list_first)
attention_masks_2 = add_attention_masks(bert_indexed_tokens_list_second)

#**Section 4: Load BERT**

In [0]:
# Convert inputs to PyTorch tensors
tokens_tensor_1 = torch.tensor([bert_indexed_tokens_list_first])
tokens_tensor_2 = torch.tensor([bert_indexed_tokens_list_second])
segments_tensors_1 = torch.tensor([attention_masks_1])
segments_tensors_2 = torch.tensor([attention_masks_2])

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-cased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()


100%|██████████| 404400730/404400730 [00:09<00:00, 43313603.54B/s]


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [0]:
# # Convert inputs to PyTorch tensors
# tokens_tensor = torch.tensor([bert_indexed_tokens_list])
# segments_tensors = torch.tensor([attention_masks])

# # Load pre-trained model (weights)
# model = BertModel.from_pretrained('bert-base-cased')

# # Put the model in "evaluation" mode, meaning feed-forward operation.
# model.eval()


In [0]:
# print(tokens_tensor.size())
# print(segments_tensors.size())

In [0]:
# tokens_tensor = tokens_tensor.view(21501, MAX_SEQ_LEN)
# segments_tensors = segments_tensors.view(21501, MAX_SEQ_LEN)
# print(tokens_tensor.size())
# print(segments_tensors.size())

In [0]:
tokens_tensor_1 = tokens_tensor_1.view(10750, MAX_SEQ_LEN)
segments_tensors_1 = segments_tensors_1.view(10750, MAX_SEQ_LEN)
tokens_tensor_2 = tokens_tensor_2.view(10751, MAX_SEQ_LEN)
segments_tensors_2 = segments_tensors_2.view(10751, MAX_SEQ_LEN)
print(tokens_tensor_1.size())
print(segments_tensors_1.size())
print(tokens_tensor_2.size())
print(segments_tensors_2.size())

torch.Size([10750, 35])
torch.Size([10750, 35])
torch.Size([10751, 35])
torch.Size([10751, 35])


What if we simply ran the below cell twice with both of the things and then appended them? 

Add batching if ram overflows!

In [0]:
embed_batch_size = 500
all_bert_embeddings_1 = []
len_dataset = len(bert_indexed_tokens_list_first)
for i in range(0, len_dataset, embed_batch_size):
  print("Batch up until:", i + embed_batch_size)
  with torch.no_grad():
    encoded_layers_1, _ = model(tokens_tensor_1[i:min(len_dataset, i + embed_batch_size)], segments_tensors_1[i:min(len_dataset, i + embed_batch_size)])
    all_bert_embeddings_1.append(encoded_layers_1)

Batch up until: 500
Batch up until: 1000
Batch up until: 1500
Batch up until: 2000
Batch up until: 2500
Batch up until: 3000
Batch up until: 3500
Batch up until: 4000
Batch up until: 4500
Batch up until: 5000
Batch up until: 5500
Batch up until: 6000
Batch up until: 6500
Batch up until: 7000
Batch up until: 7500
Batch up until: 8000
Batch up until: 8500
Batch up until: 9000
Batch up until: 9500
Batch up until: 10000
Batch up until: 10500
Batch up until: 11000


In [0]:
# embed_batch_size = 500
# all_bert_embeddings_2 = []
# len_dataset = len(bert_indexed_tokens_list_second)
# for i in range(0, len_dataset, embed_batch_size):
#   print("Batch up until:", i + embed_batch_size)
#   with torch.no_grad():
#     encoded_layers_2, _ = model(tokens_tensor_2[i:min(len_dataset, i + embed_batch_size)], segments_tensors_2[i:min(len_dataset, i + embed_batch_size)])
#     all_bert_embeddings_2.append(encoded_layers_2)

In [0]:
# embed_batch_size = 1000
# all_bert_embeddings = []
# len_dataset = len(bert_indexed_tokens_list)
# for i in range(0, len_dataset, embed_batch_size):
#   print("Batch up until:", i + embed_batch_size)
#   with torch.no_grad():
#     encoded_layers, _ = model(tokens_tensor[i:min(len_dataset, i + embed_batch_size)], segments_tensors[i:min(len_dataset, i + embed_batch_size)])
#     all_bert_embeddings.append(encoded_layers)

In [0]:
print ("Number of batches:", len(all_bert_embeddings_1))
layer_i = 0
print ("Number of layers:", len(all_bert_embeddings_1[layer_i]))
sentence_i = 0
print ("Number of sentences:", len(all_bert_embeddings_1[layer_i][sentence_i]))
token_i = 0
print ("Number of tokens in each sentence:", len(all_bert_embeddings_1[layer_i][sentence_i][token_i]))
embedding_i = 0
print ("Number of ints per token embedding:", len(all_bert_embeddings_1[layer_i][sentence_i][token_i][embedding_i]))

Number of batches: 22
Number of layers: 12
Number of sentences: 500
Number of tokens in each sentence: 35
Number of ints per token embedding: 768


Find several pooling strategies to turn into word embeddings

In [0]:
# `encoded_layers` is a Python list.
print('     Type of : ', type(all_bert_embeddings_1))

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', all_bert_embeddings_1[0][0].size())

     Type of :  <class 'list'>
Tensor shape for each layer:  torch.Size([500, 35, 768])


In [0]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
stacked_embeds = []
for batch in all_bert_embeddings_1:
  for sent in batch:
    # Swap dimensions 0 and 1.
    token_embeddings_2 = sent.permute(1,0,2)
    print(token_embeddings_2.size())
    stacked_embeds.append(token_embeddings_2)

torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
torch.Size([35, 500, 768])
t

In [0]:
# Remove dimension 1, the "batches".
token_embeddings_1 = torch.squeeze(token_embeddings, dim=1)

token_embeddings_1.size()

torch.Size([12, 250, 35, 768])

In [0]:
# Swap dimensions 0 and 1.
token_embeddings_2 = token_embeddings_1.permute(1,2,0,3)

token_embeddings_2.size()

torch.Size([250, 35, 12, 768])

In [0]:
# torch.save(token_embeddings_2, 'token_embeddings_2.pt')

In [0]:
# from google.colab import files
# files.download('token_embeddings_2.pt') 

In [0]:
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum = []

# `token_embeddings` is a [ x 12 x 768] tensor.

# For each token in the sentence...
for batch in token_embeddings_2:
  print(len(batch))
  for token in batch:

      # `token` is a [12 x 768] tensor

      # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
      
      # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
Shape is: 8750 x 768


In [0]:
print('hi')

hi


In [0]:
# # Stores the token vectors, with shape [22 x 3,072]
# token_vecs_cat = []

# # `token_embeddings` is a [22 x 12 x 768] tensor.

# # For each token in the sentence...
# #for batch in token_embeddings
# #for whatever in batch
# for token in token_embeddings:
    
#     # `token` is a [12 x 768] tensor

#     # Concatenate the vectors (that is, append them together) from the last 
#     # four layers.
#     # Each layer vector is 768 values, so `cat_vec` is length 3,072.
#     cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    
#     # Use `cat_vec` to represent `token`.
#     token_vecs_cat.append(cat_vec)

Make sure tokens align

Get features per token

Append to word embeddings, dimension should end in ~800

Get IO (not BIO) labels! Make this reproducible for devset

Pass through model, don't do test train split

Do same thing all over again to devset